In [ ]:
knitr::opts_chunk$set(echo = TRUE)



# 1. 데이터 전처리 

 - 파일명: [data_cleaning.R](https://github.com/Unfinishedgod/dacon_jeju/blob/main/data_cleaning.R)


In [ ]:
# 제주도 공간정보 탐색적 데이터 분석 경진대회
# 데이터 cleaning
library(data.table)
library(tidyverse)
library(lubridate)
library(fs)
library(sp)
library(sf)
library(rgeos)
library(rgdal)
library(leaflet)
library(plotly)
library(formattable)
library(shiny)
library(leaflet)
library(DT)
library(RColorBrewer)

# 데이콘 제주도 데이터 하나의 데이터 프레임으로 병합
data_jeju <- map_dfr(dir_ls("data", glob = "*.txt"), function(x) {
  read_csv(x)
}) %>%
  select(-X, -Y)

# 좌표 변환기 함수 위셩도 변환용도
convertCoordSystem <- function(long, lat, from.crs, to.crs){
  xy <- data.frame(long=long, lat=lat)
  coordinates(xy) <- ~long+lat

  from.crs <- CRS(from.crs)
  from.coordinates <- SpatialPoints(xy, proj4string=from.crs)

  to.crs <- CRS(to.crs)
  changed <- as.data.frame(SpatialPoints(spTransform(from.coordinates, to.crs)))
  names(changed) <- c("long", "lat")

  return(changed)
}


# 지리정보 API를 사용한 위경도에서 주소 추출 함수
Lat_lon_to_address_fun <- function(long, lat) {
  data_list <-
    GET(url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json',
        # query = list(x = long, y= lat),
        query = list(x = long, y= lat),
        add_headers(Authorization = paste0("KakaoAK ", "2ecacbafd523802f293b245103346b06"))) %>%
    content(as = 'text') %>%
    fromJSON()

  lon_lat_df <- tibble(
    long = long,
    lat = lat,
    주소 = data_list$documents$address_name,
    구 = data_list$documents$region_2depth_name,
    동 = data_list$documents$region_3depth_name
  ) %>%
    head(1)

  return(lon_lat_df)
}


# 좌표 변환
coord <- data.frame(utmk.long=c(954677.6, 958869.4), utmk.lat=c(1951583, 1945669))
from.crs = "+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs"

to.crs = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"

# convertCoordSystem() 함수 사용하여 좌표체계와 위경도를 구성하는 df 생성, lat_long_df
lat_long_df <- convertCoordSystem(data_jeju$POINT_X, data_jeju$POINT_Y, from.crs, to.crs) %>%
  as_tibble()

# 위경도에서 주소 변환 카카오 api사용하며, 하루 할당량 약 10만개로, 위경도 unique하여 주소 변환
# 이를 lat_long_df_unique라는 df 생성
lat_long_df$long <- as.character(lat_long_df$long)
lat_long_df$lat <- as.character(lat_long_df$lat)
lat_long_df_unique <- lat_long_df %>%
  unique()

# # 위경도에서 주소로 변환시키는 Lat_lon_to_address_fun()함수를 사용하여 위경도, 주소를 담고 있는 df 변환
# # 이를 lat_long_addr_df라는 df 생성
lat_long_addr_df  <- map_dfr(1:nrow(lat_long_df_unique), function(x) {
  Lat_lon_to_address_fun(lat_long_df_unique$long[x], lat_long_df_unique$lat[x])
})

# lat_long_addr_df <- read_csv("data/long_lat_addr_matching_csv")
lat_long_addr_df$long <- lat_long_addr_df$long %>% as.character()
lat_long_addr_df$lat <- lat_long_addr_df$lat %>% as.character()

jeju_lat_long_addr_df  <- lat_long_df %>%
  left_join(lat_long_addr_df)

# 기존 data_jeju와 jeju_lat_long_addr_df와 column bind, 이를 total_df
total_df <- data_jeju %>%
  bind_cols(jeju_lat_long_addr_df)


# 2. Shiny file

## 2-1. global.R

 - 파일명: [global.R](https://github.com/Unfinishedgod/dacon_jeju/blob/main/global.R)


In [ ]:
# 제주도 공간정보 탐색적 데이터 분석 경진대회
library(tidyverse)
library(lubridate)
library(plotly)
library(formattable)
library(shiny)
library(leaflet)
library(DT)
library(RColorBrewer)
library(shinydashboard)
library(reactable)
library(rsconnect)

# law 데이터 로드
total_df <- read_csv("total_df.csv")
# total_df <- read_csv("../total_df.csv")

# Time중 "x시"는 제거
total_df <- total_df %>% 
  filter(Time != "x시") %>% 
  as_tibble()

# Franclass에 따른 컬러 코드 작성
color_index <- total_df$FranClass %>% unique()
colors_list <- brewer.pal(length(color_index), "Set1")

# Select Box ----
YM_list <- total_df$YM %>% unique()
FranClass_list <- total_df$FranClass %>% unique()
Time_list <- total_df$Time %>% unique()
Type_list <- total_df$Type %>% unique()

YM_list_2 <- c("ALL", YM_list)
FranClass_list_2 <- c("ALL", FranClass_list)
Time_list_2 <- c("ALL", Time_list)

# 시간에 따른 조정 ----
timeline_raw <- total_df %>% 
  group_by(Time, Type) %>% 
  summarise(aaa = mean(TotalSpent), 
            bbb = mean(DisSpent)) 

timeline_raw <- crossing("Time"= Time_list, "Type"= Type_list) %>% 
  left_join(timeline_raw)

timeline_raw <- timeline_raw %>% 
  group_by(Time, Type) %>% 
  summarise(aaa_sum = mean(aaa), 
            bbb_sum = mean(bbb))

timeline_raw[is.na(timeline_raw)] <- 0

timeline_total_df <- timeline_raw %>% 
  filter(Time %in% c("02시", "06시", "11시", "15시", "18시", "22시")) %>% 
  select(-bbb_sum) %>% 
  spread(key = "Time", value = "aaa_sum") %>% 
  mutate(`심야~새벽` = round((`06시` - `02시`),2), 
         `새벽~오전` = round((`11시` - `06시`),2),
         `오전~점심` = round((`15시` - `11시`),2),
         `점심~오후` = round((`18시` - `15시`),2),
         `오후~저녁` = round((`22시` - `18시`),2),
         `저녁~심야` = round((`02시` - `22시`),2))

timeline_dis_df <- timeline_raw %>% 
  filter(Time %in% c("02시", "06시", "11시", "15시", "18시", "22시")) %>% 
  select(-aaa_sum) %>% 
  spread(key = "Time", value = "bbb_sum") %>% 
  mutate(`심야~새벽` = round((`06시` - `02시`),2), 
         `새벽~오전` = round((`11시` - `06시`),2),
         `오전~점심` = round((`15시` - `11시`),2),
         `점심~오후` = round((`18시` - `15시`),2),
         `오후~저녁` = round((`22시` - `18시`),2),
         `저녁~심야` = round((`02시` - `22시`),2))


diff_time_list <- c("심야~새벽", "새벽~오전","오전~점심","점심~오후", "오후~저녁", "저녁~심야")


# vline function 
vline <- function(x = 0, color = "black") {
  list(
    type = "line",
    y0 = 0,
    y1 = 1,
    yref = "paper",
    x0 = x,
    x1 = x,
    line = list(color = color, dash = 'dot', width = 1)
  )
}

## Shiny ui, server
print("=================ui.R=================")
source("./ui.R" , local = TRUE)
print("=================server.R=================")
source("./server.R" , local = TRUE)

shinyApp(ui, server)


## 2-2. ui.R

 - 파일명: [ui.R](https://github.com/Unfinishedgod/dacon_jeju/blob/main/ui.R)


In [ ]:
# 제주도 공간정보 탐색적 데이터 분석 경진대회
print("------------ui------------")

ui <-   
  navbarPage("DACON: 공간정보 탐색적 데이터 분석 경진대회", id="nav",
             # # Main ----
             # tabPanel("Dacon",
             #          tags$a(
             #            href="https://dacon.io/competitions/official/235682/overview/", 
             #            tags$img(src="img_main.PNG", 
             #                     title="Example Image Link", 
             #                     width="100%",height="25%"))
             # ), 
             
             # Map ----
             tabPanel("소상공인 구분 지도 분포",
                      div(class="outer",
                          tags$head(
                            shiny::tags$link(rel = "shortcut icon", href = "https://unfinishedgod.netlify.app/Gareth_Frank_Bale_circle_index.ico"), 
                            # Include our custom CSS
                            includeCSS("styles.css")
                          ),
                          
                          # leaflet
                          leafletOutput("bbmap", width="100%", height="100%"),
                          
                          # Shiny versions prior to 0.11 should use class = "modal" instead.
                          absolutePanel(id = "controls", class = "panel panel-default", fixed = TRUE,
                                        draggable = TRUE, top = 60, left = "auto", right = 20, bottom = "auto",
                                        width = 450, height = 1000,
                                        
                                        h2("Option"),
                                        h4("총 1,072,439개중 표본으로 약 10724개 사용"), 
                                        
                                        selectInput(inputId = "selectTime",
                                                    label = h4("시간"),
                                                    choices = Time_list_2,
                                                    selected = "ALL",
                                                    width = 400),
                                        
                                        selectInput(inputId = "selectYM",
                                                    label = h4("년/월"),
                                                    choices = YM_list_2,
                                                    selected = "ALL",
                                                    width = 400),
                                        
                                        selectInput(inputId = "selectFranClass",
                                                    label = h4("소상공인구분"),
                                                    choices = FranClass_list_2,
                                                    selected = "ALL",
                                                    width = 400),
                                        
                                        
                                        
                                        plotlyOutput("histCentile", height = 300),
                                        plotlyOutput("scatterCollegeIncome", height = 300)
                                        
                          )
                      )
             ),
             
             # 시간에 따른 업종별 총 사용금액----
             tabPanel("총사용 사용금액",
                      fluidRow(
                        h3("시간에 따른 총 사용금액 증감 현황"), 
                        box(
                          width = 4, status = "info", solidHeader = TRUE,
                          selectInput(inputId = "selectType_2",
                                      label = h4("업종명"),
                                      choices = diff_time_list,
                                      selected = c("면세점","항공사","인터넷Mall","대형할인점","농협하나로클럽","건강진단","농축협직영매장","종합병원"),
                                      width = 500)
                        ),
                        box(
                        )
                        
                      ),
                      fluidRow(
                        h3("시간에 따른 총 사용금액 증가 상위 10개 업종"), 
                        box(
                          width = 9, status = "info", solidHeader = TRUE,
                          plotlyOutput("total_spend_increase")
                        ),
                        box(
                          width = 3, status = "info", solidHeader = TRUE,
                          reactableOutput("total_spend_increase_table")
                        )
                      ),
                      fluidRow(
                        h3("시간에 따른 총 사용금액 감소 하위 10개 업종"), 
                        box(
                          width = 9, status = "info", solidHeader = TRUE,
                          plotlyOutput("total_spend_decrease")
                        ),
                        box(
                          width = 3, status = "info", solidHeader = TRUE,
                          reactableOutput("total_spend_decrease_table")
                        )
                        
                      )
             ), 
             
             # 시간에 따른 업종별 재난 지원금 사용금액----
             tabPanel("재난지원금 사용금액",
                      fluidRow(
                        h3("시간에 따른 재난지원금 사용금액 증감 현황"), 
                        box(
                          width = 4, status = "info", solidHeader = TRUE,
                          selectInput(inputId = "selectType_3",
                                      label = h4("업종명"),
                                      choices = diff_time_list,
                                      selected = c("면세점","항공사","인터넷Mall","대형할인점","농협하나로클럽","건강진단","농축협직영매장","종합병원"),
                                      width = 500)
                        ),
                        box(
                        )
                      ),
                      fluidRow(
                        h3("시간에 따른 재난지원금 사용금액 증가 상위 10개 업종"), 
                        box(
                          width = 9, status = "info", solidHeader = TRUE,
                          plotlyOutput("dis_spend_increase")
                        ),
                        box(
                          width = 3, status = "info", solidHeader = TRUE,
                          reactableOutput("dis_spend_increase_table")
                        )
                      ),
                      fluidRow(
                        h3("시간에 따른 재난지원금 사용금액 감소 하위 10개 업종"), 
                        box(
                          width = 9, status = "info", solidHeader = TRUE,
                          plotlyOutput("dis_spend_decrease")
                        ),
                        box(
                          width = 3, status = "info", solidHeader = TRUE,
                          reactableOutput("dis_spend_decrease_table")
                        )
                      ), 
             ),
             
             # Data ----
             tabPanel("데이터",
                      DT::dataTableOutput("data")
             )
             # # About ----
             # tabPanel("About",
             # )
  )
# )


## 2-3. server.R

 - 파일명: [server.R](https://github.com/Unfinishedgod/dacon_jeju/blob/main/server.R)
 


In [ ]:
# 제주도 공간정보 탐색적 데이터 분석 경진대회

print("------------server------------")


server <- shinyServer(function(input, output) {
  
  # leaflet 반응형 데이터 ----
  leaflet_data <- reactive({
    
    total_df <- total_df[sample(1:nrow(total_df), nrow(total_df)/100), ]
    
    
    if(input$selectTime == "ALL") {
      fixed_selectTime <- Time_list
    } else {
      fixed_selectTime <- input$selectTime 
    }
    
    if(input$selectYM == "ALL") {
      fixed_selectYM <- YM_list
    } else {
      fixed_selectYM <- input$selectYM
    }
    
    if(input$selectFranClass == "ALL") {
      fixed_selectFranClass <- FranClass_list
    } else {
      fixed_selectFranClass <- input$selectFranClass
    }

    color_index <-  total_df$FranClass %>% unique()
    colors_list <- brewer.pal(length(color_index), "Set1")
    
    total_df <- total_df %>% 
      mutate(color = case_when(
        FranClass == color_index[1] ~ colors_list[1], 
        FranClass == color_index[2] ~ colors_list[2], 
        FranClass == color_index[3] ~ colors_list[3], 
        FranClass == color_index[4] ~ colors_list[4], 
        TRUE ~ colors_list[5]
      ))
    
    
    total_df %>%
      filter(Time %in% fixed_selectTime) %>%
      filter(YM %in% fixed_selectYM) %>%
      filter(FranClass %in% fixed_selectFranClass)
  })
  

  
  # Map Tab ----
  # _leaflet 제주도  ----
  output$bbmap <- renderLeaflet({
    
    test_df <- leaflet_data()
    
    # # Franclass에 따른 컬러 코드 작성
    # color_index_leaflet <- test_df$FranClass %>% unique()
    # colors_list_leaflet <- brewer.pal(length(color_index_leaflet), "Set1")
    
    
    leaflet() %>%
      setView(lng = 126.7229, lat = 33.40035, zoom = 11) %>% 
      addTiles() %>%
      addCircleMarkers(test_df$long,
                       test_df$lat,
                       color = test_df$color,
                       radius = 3,
                       fill = T,
                       fillOpacity = 1,
                       opacity = 0.5,
                       popup = paste0("소상공인구분: ",test_df$FranClass,"",
                                      "<br/> 업종명: ", test_df$Type,
                                      "<br/> 기준년월: ", test_df$YM, 
                                      "<br/> 시간대: ", test_df$Time,
                                      "<br/> 주소: ", test_df$주소, 
                                      "<br/> 총사용금액(재난지원금 포함): ",round(comma(test_df$TotalSpent , 0), 0) ,"",
                                      "<br/> 재난지원금 사용금액: ", round(comma(test_df$DisSpent , 0), 0),"")) %>%
      addLegend("topleft",
                colors = unique(test_df$color),
                labels = unique(test_df$FranClass),
                opacity = 1)
  })
  
  # _Type별 총 금액 ----
  output$histCentile <- renderPlotly({
    leaflet_data() %>% 
      group_by(Type) %>% 
      summarise(TotalSpent_1 = mean(TotalSpent),
                DisSpent_1 = mean(DisSpent)) %>% 
      arrange(desc(TotalSpent_1)) %>% 
      head(5) %>% 
      plot_ly() %>% 
      add_trace(x = ~TotalSpent_1, y = ~Type, type = 'bar', 
                text = ~paste0(round(comma(TotalSpent_1),0), "원"),
                hoverinfo = "text",
                hovertemplate = ~paste0(Type, "\n", comma(TotalSpent_1), "원"),
                textposition = "inside",
                marker = list(color = colors_list)
                
      ) %>%
      layout(
        title = "업종별 사용금액",
        xaxis = list(title = ""),
        yaxis = list(title = "",
                     categoryorder = "array",
                     categoryarray = ~TotalSpent_1,
                     size=5),
        plot_bgcolor  = "rgba(0, 0, 0, 0)",
        paper_bgcolor = "rgba(0, 0, 0, 0)",
        fig_bgcolor   = "rgba(0, 0, 0, 0)"
      )
  })
  
  # _Type별 재난지원금 사용 금액 ----
  output$scatterCollegeIncome <- renderPlotly({
    leaflet_data() %>% 
      group_by(Type) %>% 
      summarise(TotalSpent_1 = mean(TotalSpent),
                DisSpent_1 = mean(DisSpent)) %>% 
      arrange(desc(DisSpent_1)) %>% 
      head(5) %>% 
      plot_ly() %>% 
      add_trace(x = ~DisSpent_1, y = ~Type, 
                text = ~paste0(round(comma(DisSpent_1),0), "원"),
                hoverinfo = "text",
                hovertemplate = ~paste0(Type, "\n", comma(DisSpent_1), "원"),
                textposition = "inside" ,
                marker = list(color = colors_list)
      ) %>% 
      layout(
        title = "업종별 재난지원금 사용금액",
        xaxis = list(title = ""),
        yaxis = list(title = "",
                     categoryorder = "array",
                     categoryarray = ~DisSpent_1,
                     size=5),
        plot_bgcolor  = "rgba(0, 0, 0, 0)",
        paper_bgcolor = "rgba(0, 0, 0, 0)",
        fig_bgcolor   = "rgba(0, 0, 0, 0)"
      ) 
  })
  
  # 시각화 Tab ----
  
  # leaflet 반응형 데이터 ----
  vis_data <- reactive({
    total_df
  })
  
  # _시간에 따른 업종별 총 사용금액 증가  ----
  output$total_spend_increase <- renderPlotly({
    
    if(input$selectType_2 %in% diff_time_list) {
      fix_selectType <- timeline_total_df %>% 
        select(Type, input$selectType_2) %>%
        rename(`증감` = input$selectType_2) %>% 
        arrange(desc(`증감`)) %>% 
        head(10) %>% 
        select(Type) %>% 
        t() %>% 
        as.character()
    } else {
      fix_selectType <- input$selectType_2
    }
    
    # leaflet_data()
    type_time_df <- total_df %>%
      filter(Type %in% fix_selectType) %>% 
      group_by(Time, Type) %>% 
      summarise(TotalSpent_mean = mean(TotalSpent), 
                DisSpent_mean = mean(DisSpent))
    
    type_time_df <- crossing("Time"= Time_list, "Type"= fix_selectType) %>% 
      left_join(type_time_df) %>% 
      fill(Type) %>% 
      replace_na(list(TotalSpent_mean = 0, DisSpent_mean = 0))
    
    type_time_df %>% 
      plot_ly() %>% 
      add_trace(x = ~Time, y = ~TotalSpent_mean, type="scatter", mode="line", color = ~Type, line = list(shape = "spline"),
                text = ~paste0(round(comma(TotalSpent_mean),0), "원"),
                hoverinfo = "text",
                hovertemplate = ~paste0(Time, "\n", Type, "\n", comma(TotalSpent_mean), "원"),
                textposition = "inside"
      ) %>% 
      layout(shapes = list(vline("02시"), 
                           vline("06시"), 
                           vline("11시"), 
                           vline("15시"), 
                           vline("18시"), 
                           vline("22시"))) %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "04시",
          y = max(type_time_df$TotalSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "새벽",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      ) %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "08시",
          y = max(type_time_df$TotalSpent_mean),
          xanchor = 'left',
          yanchor = 'top',
          text = "오전",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      ) %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "13시",
          y = max(type_time_df$TotalSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "점심",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )  %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "16시",
          y = max(type_time_df$TotalSpent_mean),
          xanchor = 'left',
          yanchor = 'top',
          text = "오후",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )  %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "20시",
          y = max(type_time_df$TotalSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "저녁",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )  %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "00시",
          y = max(type_time_df$TotalSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "심야",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )

    
  })
  
  # _시간에 따른 업종별 총 사용금 증가 비율 ----
  output$total_spend_increase_table <- renderReactable({
    if(input$selectType_2 %in% diff_time_list) {
      fix_selectType <- timeline_total_df %>% 
        select(Type, input$selectType_2) %>%
        rename(`증감` = input$selectType_2) %>% 
        arrange(desc(`증감`)) %>% 
        head(10) %>% 
        select(Type) %>% 
        t() %>% 
        as.character()
    } else {
      fix_selectType <- input$selectType_2
    }
    
    timeline_total_df %>% 
      select(Type, input$selectType_2) %>%
      rename(`증감` = input$selectType_2) %>% 
      arrange(desc(`증감`)) %>% 
      head(10) %>% 
      reactable(
        columns = list(
        `증감` = colDef(aggregate = "sum", format = colFormat(separators = TRUE))
      ))
  })
  
  
  # _시간에 따른 업종별 총 사용금액 감소  ----
  output$total_spend_decrease <- renderPlotly({
    
    if(input$selectType_2 %in% diff_time_list) {
      fix_selectType <- timeline_total_df %>% 
        select(Type, input$selectType_2) %>%
        rename(`증감` = input$selectType_2) %>% 
        arrange((`증감`)) %>% 
        head(10) %>% 
        select(Type) %>% 
        t() %>% 
        as.character()
    } else {
      fix_selectType <- input$selectType_2
    }
    
    # leaflet_data()
    type_time_df <- total_df %>%
      filter(Type %in% fix_selectType) %>% 
      group_by(Time, Type) %>% 
      summarise(TotalSpent_mean = mean(TotalSpent), 
                DisSpent_mean = mean(DisSpent))
    
    type_time_df <- crossing("Time"= Time_list, "Type"= fix_selectType) %>% 
      left_join(type_time_df) %>% 
      fill(Type) %>% 
      replace_na(list(TotalSpent_mean = 0, DisSpent_mean = 0))
    
    type_time_df %>% 
      plot_ly() %>% 
      add_trace(x = ~Time, y = ~TotalSpent_mean, type="scatter", mode="line", color = ~Type, line = list(shape = "spline"),
                text = ~paste0(round(comma(TotalSpent_mean),0), "원"),
                hoverinfo = "text",
                hovertemplate = ~paste0(Time, "\n", Type, "\n", comma(TotalSpent_mean), "원"),
                textposition = "inside"
      ) %>% 
      layout(shapes = list(vline("02시"), 
                           vline("06시"), 
                           vline("11시"), 
                           vline("15시"), 
                           vline("18시"), 
                           vline("22시"))) %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "04시",
          y = max(type_time_df$TotalSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "새벽",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      ) %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "08시",
          y = max(type_time_df$TotalSpent_mean),
          xanchor = 'left',
          yanchor = 'top',
          text = "오전",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      ) %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "13시",
          y = max(type_time_df$TotalSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "점심",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )  %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "16시",
          y = max(type_time_df$TotalSpent_mean),
          xanchor = 'left',
          yanchor = 'top',
          text = "오후",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )  %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "20시",
          y = max(type_time_df$TotalSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "저녁",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )  %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "00시",
          y = max(type_time_df$TotalSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "심야",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )
    
    
  })
  
  # _시간에 따른 업종별 총 사용금 감소 비율 ----
  output$total_spend_decrease_table <- renderReactable({
    if(input$selectType_2 %in% diff_time_list) {
      fix_selectType <- timeline_total_df %>% 
        select(Type, input$selectType_2) %>%
        rename(`증감` = input$selectType_2) %>% 
        arrange((`증감`)) %>% 
        head(10) %>% 
        select(Type) %>% 
        t() %>% 
        as.character()
    } else {
      fix_selectType <- input$selectType_2
    }
    
    
    timeline_total_df %>% 
      select(Type, input$selectType_2) %>%
      rename(`증감` = input$selectType_2) %>% 
      arrange((`증감`)) %>% 
      head(10) %>% 
      reactable(
        columns = list(
          `증감` = colDef(aggregate = "sum", format = colFormat(separators = TRUE))
        ))
  })
  
  

  
  # _시간에 따른 업종별 재난지원금 사용금액 증가  ----
  output$dis_spend_increase <- renderPlotly({
    
    if(input$selectType_3 %in% diff_time_list) {
      fix_selectType <- timeline_dis_df %>% 
        select(Type, input$selectType_3) %>%
        rename(`증감` = input$selectType_3) %>% 
        arrange(desc(`증감`)) %>% 
        head(10) %>% 
        select(Type) %>% 
        t() %>% 
        as.character()
    } else {
      fix_selectType <- input$selectType_3
    }
    
    # leaflet_data()
    type_time_df <- total_df %>%
      filter(Type %in% fix_selectType) %>% 
      group_by(Time, Type) %>% 
      summarise(TotalSpent_mean = mean(TotalSpent), 
                DisSpent_mean = mean(DisSpent))
    
    type_time_df <- crossing("Time"= Time_list, "Type"= fix_selectType) %>% 
      left_join(type_time_df) %>% 
      fill(Type) %>% 
      replace_na(list(TotalSpent_mean = 0, DisSpent_mean = 0))
    
    type_time_df %>% 
      plot_ly() %>% 
      add_trace(x = ~Time, y = ~DisSpent_mean, type="scatter", mode="line", color = ~Type, line = list(shape = "spline"),
                text = ~paste0(round(comma(DisSpent_mean),0), "원"),
                hoverinfo = "text",
                hovertemplate = ~paste0(Time, "\n", Type, "\n", comma(DisSpent_mean), "원"),
                textposition = "inside"
      ) %>% 
      layout(shapes = list(vline("02시"), 
                           vline("06시"), 
                           vline("11시"), 
                           vline("15시"), 
                           vline("18시"), 
                           vline("22시"))) %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "04시",
          y = max(type_time_df$DisSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "새벽",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      ) %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "08시",
          y = max(type_time_df$DisSpent_mean),
          xanchor = 'left',
          yanchor = 'top',
          text = "오전",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      ) %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "13시",
          y = max(type_time_df$DisSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "점심",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )  %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "16시",
          y = max(type_time_df$DisSpent_mean),
          xanchor = 'left',
          yanchor = 'top',
          text = "오후",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )  %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "20시",
          y = max(type_time_df$DisSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "저녁",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )  %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "00시",
          y = max(type_time_df$DisSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "심야",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )
    
    
  })
  
  # _시간에 따른 업종별 재난지원금 사용금 증가 비율 ----
  output$dis_spend_increase_table <- renderReactable({
    if(input$selectType_3 %in% diff_time_list) {
      fix_selectType <- timeline_dis_df %>% 
        select(Type, input$selectType_3) %>%
        rename(`증감` = input$selectType_3) %>% 
        arrange(desc(`증감`)) %>% 
        head(10) %>% 
        select(Type) %>% 
        t() %>% 
        as.character()
    } else {
      fix_selectType <- input$selectType_3
    }
    
    timeline_dis_df %>% 
      select(Type, input$selectType_3) %>%
      rename(`증감` = input$selectType_3) %>% 
      arrange(desc(`증감`)) %>% 
      head(10) %>% 
      reactable(
        columns = list(
          `증감` = colDef(aggregate = "sum", format = colFormat(separators = TRUE))
        ))
  })
  
  
  # _시간에 따른 업종별 재난지원금 사용금액 감소  ----
  output$dis_spend_decrease <- renderPlotly({
    
    if(input$selectType_3 %in% diff_time_list) {
      fix_selectType <- timeline_dis_df %>% 
        select(Type, input$selectType_3) %>%
        rename(`증감` = input$selectType_3) %>% 
        arrange((`증감`)) %>% 
        head(10) %>% 
        select(Type) %>% 
        t() %>% 
        as.character()
    } else {
      fix_selectType <- input$selectType_3
    }
    
    # leaflet_data()
    type_time_df <- total_df %>%
      filter(Type %in% fix_selectType) %>% 
      group_by(Time, Type) %>% 
      summarise(TotalSpent_mean = mean(TotalSpent), 
                DisSpent_mean = mean(DisSpent))
    
    type_time_df <- crossing("Time"= Time_list, "Type"= fix_selectType) %>% 
      left_join(type_time_df) %>% 
      fill(Type) %>% 
      replace_na(list(TotalSpent_mean = 0, DisSpent_mean = 0))
    
    type_time_df %>% 
      plot_ly() %>% 
      add_trace(x = ~Time, y = ~DisSpent_mean, type="scatter", mode="line", color = ~Type, line = list(shape = "spline"),
                text = ~paste0(round(comma(DisSpent_mean),0), "원"),
                hoverinfo = "text",
                hovertemplate = ~paste0(Time, "\n", Type, "\n", comma(DisSpent_mean), "원"),
                textposition = "inside"
      ) %>% 
      layout(shapes = list(vline("02시"), 
                           vline("06시"), 
                           vline("11시"), 
                           vline("15시"), 
                           vline("18시"), 
                           vline("22시"))) %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "04시",
          y = max(type_time_df$DisSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "새벽",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      ) %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "08시",
          y = max(type_time_df$DisSpent_mean),
          xanchor = 'left',
          yanchor = 'top',
          text = "오전",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      ) %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "13시",
          y = max(type_time_df$DisSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "점심",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )  %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "16시",
          y = max(type_time_df$DisSpent_mean),
          xanchor = 'left',
          yanchor = 'top',
          text = "오후",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )  %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "20시",
          y = max(type_time_df$DisSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "저녁",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )  %>% 
      layout(
        annotations = list(
          xref = 'x',
          yref = 'y',
          x = "00시",
          y = max(type_time_df$DisSpent_mean),
          xanchor = 'meddle',
          yanchor = 'top',
          text = "심야",
          font = list(family = 'Arial',
                      size = 16,
                      color = 'rgba(67,67,67,1)'),
          showarrow = FALSE)
      )
    
    
  })
  
  # _시간에 따른 업종별 재난지원금 사용금 감소 비율 ----
  output$dis_spend_decrease_table <- renderReactable({
    if(input$selectType_3 %in% diff_time_list) {
      fix_selectType <- timeline_dis_df %>% 
        select(Type, input$selectType_3) %>%
        rename(`증감` = input$selectType_3) %>% 
        arrange((`증감`)) %>% 
        head(10) %>% 
        select(Type) %>% 
        t() %>% 
        as.character()
    } else {
      fix_selectType <- input$selectType_3
    }
    
    
    timeline_dis_df %>% 
      select(Type, input$selectType_3) %>%
      rename(`증감` = input$selectType_3) %>% 
      arrange((`증감`)) %>% 
      head(10) %>% 
      reactable(
        columns = list(
          `증감` = colDef(aggregate = "sum", format = colFormat(separators = TRUE))
        ))
  })
  
  
  
  
  # Row data ----
  output$data <- renderDataTable({
    leaflet_data() %>%
      select(-OBJECTID, -Field1) %>% 
      DT::datatable(
        filter = 'top',
        escape = FALSE, 
        selection = 'none', 
        rownames = FALSE,
        options = list(
          pageLength =20
        )
      )
  })
  
})


 
# 3. styile.css

 - 파일명: [style.css](https://github.com/Unfinishedgod/dacon_jeju/blob/main/styles.css)

css파일은 git주소로 대체
 
 
